In [3]:
from src.tools.seo_tools import SeoTools

seo_tools = await SeoTools().get_all_tools()

In [43]:
from typing import Any, Dict, List
import json


def infer_server_from_tool(tool: Any) -> str:
    """
    Robustly infer whether this tool belongs to the GSC or DataForSEO MCP server.
    Priority:
    1) tool.metadata["mcp_server"] (or similar)
    2) name / description hints
    3) FINAL FALLBACK: treat as dataforseo if it's not clearly GSC
    """
    name = getattr(tool, "name", "") or ""
    desc = getattr(tool, "description", "") or ""
    lower_name = name.lower()
    lower_desc = desc.lower()

    # 1) Try metadata first
    meta = getattr(tool, "metadata", {}) or {}
    if isinstance(meta, dict):
        # Adapt this key if your adapter uses a different one
        mcp_server = meta.get("mcp_server") or meta.get("server") or meta.get("source")
        if isinstance(mcp_server, str):
            ms = mcp_server.lower()
            if "gsc" in ms or "searchconsole" in ms:
                return "gsc"
            if "dataforseo" in ms or "dfs" in ms:
                return "dataforseo"

    # 2) Name/description heuristics

    # Strong hints for GSC
    if (
        "search console" in lower_desc
        or "search console" in lower_name
        or "gsc" in lower_name
        or "gsc" in lower_desc
        or "site" in lower_name and "search console" in lower_desc
    ):
        return "gsc"

    # Strong hints for DataForSEO
    if (
        "dataforseo" in lower_desc
        or "dataforseo" in lower_name
        or "dfs" in lower_name
        or "dfs" in lower_desc
    ):
        return "dataforseo"

    # 3) Fallback: since you only have TWO servers, if it's not clearly GSC,
    # we treat it as DataForSEO.
    # This guarantees DataForSEO tools are NOT dropped as "unknown".
    if any(k in lower_desc for k in ["property", "search console", "coverage", "impressions", "clicks"]):
        return "gsc"

    # Everything else → dataforseo
    return "dataforseo"


def infer_category_from_tool(tool: Any, server: str) -> List[str]:
    """
    Assign 1+ SEO categories based on name/description.
    We are intentionally generous for DataForSEO so NO tool is category-less.
    """
    name = getattr(tool, "name", "") or ""
    desc = getattr(tool, "description", "") or ""
    text = f"{name} {desc}".lower()

    categories: List[str] = []

    # ---------------------- GSC categories ----------------------
    if server == "gsc":
        if any(k in text for k in ["search_analytics", "analytics", "performance", "traffic", "clicks", "impressions"]):
            categories.append("gsc_performance")

        if any(k in text for k in ["query", "queries"]):
            categories.append("gsc_queries")

        if any(k in text for k in ["page", "pages", "url", "urls"]):
            categories.append("gsc_pages")

        if any(k in text for k in ["property", "properties", "site", "sites"]):
            categories.append("gsc_properties")

        if any(k in text for k in ["coverage", "index", "indexing", "sitemap"]):
            categories.append("technical_audit")

    # ------------------- DataForSEO categories -------------------
    if server == "dataforseo":
        # keywords
        if "keyword" in text:
            categories.append("keywords")

        # SERP-related
        if "serp" in text or "organic results" in text or "search results" in text:
            categories.append("serp")

        # backlinks
        if "backlink" in text or "referring" in text:
            categories.append("backlinks")

        # rank tracking / positions
        if "rank" in text or "position" in text:
            categories.append("rank_tracking")

        # ads / cpc / paid search
        if any(k in text for k in ["cpc", "paid", "ads", "ppc"]):
            categories.append("paid_search")

        # technical / on-page audit
        if any(k in text for k in ["audit", "onpage", "on-page", "site audit"]):
            categories.append("technical_audit")

        # traffic / domain-level data
        if any(k in text for k in ["traffic", "visits", "domain overview", "domain_info"]):
            categories.append("domain_insights")

    # ------------------- Fallback categories --------------------
    if not categories:
        if server == "gsc":
            categories.append("gsc_misc")
        elif server == "dataforseo":
            categories.append("dataforseo_misc")

    return categories



def build_seo_category_tool_hints(tools: List[Any]) -> Dict[str, Dict[str, List[str]]]:
    """
    Given a list of StructuredTool, build a SEO_CATEGORY_TOOL_HINTS-style mapping:

    {
        "gsc_performance": {
            "gsc": ["get_search_analytics", ...]
        },
        "keywords": {
            "dataforseo": ["get_keyword_overview", ...]
        },
        ...
    }

    With the new logic, EVERY tool should appear in at least one category.
    """
    hints: Dict[str, Dict[str, List[str]]] = {}

    for tool in tools:
        name = getattr(tool, "name", "") or ""
        if not name:
            continue

        server = infer_server_from_tool(tool)
        # We now NEVER return "unknown", but keep this guard just in case
        if server not in ("gsc", "dataforseo"):
            # print(f"Skipping tool with unknown server: {name}")
            continue

        categories = infer_category_from_tool(tool, server)

        for category in categories:
            if category not in hints:
                hints[category] = {}
            if server not in hints[category]:
                hints[category][server] = []

            if name not in hints[category][server]:
                hints[category][server].append(name)

    return hints


def debug_print_seo_category_tool_hints(tools: List[Any]) -> None:
    hints = build_seo_category_tool_hints(tools)
    
    # print the hints in a md file
    with open("seo_category_tool_hints.json", "w") as f:
        f.write(json.dumps(hints, indent=2))


In [44]:
debug_print_seo_category_tool_hints(seo_tools)

In [40]:
# put the ouput of seo_tools in a md file and save it
with open("seo_tools.md", "w") as f:
    f.write(str(seo_tools[:5]))

In [33]:
import json
from langchain_openai import ChatOpenAI
from src.tools.seo_tools import SeoTools
from src.agents.seo_agent import SEOAgent

llm = ChatOpenAI(model="gpt-4.1", temperature=0)
seo_tools = SeoTools()
seo_agent = SEOAgent(llm, seo_tools)

user_query = "How can I increase organic keyword coverage for wyo.in using GSC + keyword research?"
plan = await seo_agent.plan_query(user_query)

plan = await seo_agent.plan_query(user_query)

print(json.dumps(plan.dict(), indent=2))

{
  "original_query": "How can I increase organic keyword coverage for wyo.in using GSC + keyword research?",
  "summary": "This plan combines Google Search Console data with external keyword research to identify gaps and opportunities for expanding organic keyword coverage for wyo.in. It involves analyzing current performance, researching new keywords, comparing coverage, and prioritizing actions.",
  "steps": [
    {
      "id": 1,
      "goal": "Extract current organic keyword data (queries, impressions, clicks, positions) for wyo.in to understand existing coverage.",
      "server": "gsc",
      "categories": [
        "gsc_performance",
        "gsc_queries"
      ],
      "required_inputs": [
        "domain",
        "date_range"
      ],
      "notes": "Focus on non-branded queries if possible to maximize new keyword opportunities."
    },
    {
      "id": 2,
      "goal": "Conduct comprehensive keyword research to identify relevant keywords not currently targeted or ranking f

/var/folders/l9/vn_czxl100v8f8l1v4d02fx40000gp/T/ipykernel_82054/490424049.py:15: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(json.dumps(plan.dict(), indent=2))


In [ ]:
from langchain_openai import ChatOpenAI
from src.tools.seo_tools import SeoTools
from src.agents.seo_agent import SEOAgent

llm = ChatOpenAI(model="gpt-4.1", temperature=0)
seo_tools = SeoTools()
seo_agent = SEOAgent(llm, seo_tools)

user_query = "Provide SEO analysis for domain strique.io and suggest keyword expansion opportunities."
plan = await seo_agent.plan_query(user_query)

tool_selection = await seo_agent.select_tools_for_plan(plan)

import json
print(json.dumps(tool_selection.dict(), indent=2))

In [2]:
from langchain_openai import ChatOpenAI
from src.tools.seo_tools import SeoTools
from src.agents.seo_agent import SEOAgent

llm = ChatOpenAI(model="gpt-4.1", temperature=0)
seo_tools = SeoTools()
seo_agent = SEOAgent(llm, seo_tools)

user_query = "Provide SEO analysis for domain strique.io and suggest keyword expansion opportunities."
plan = await seo_agent.plan_query(user_query)

tool_selection = await seo_agent.select_tools_for_plan(plan)

import json
print(json.dumps(tool_selection.dict(), indent=2))


{
  "original_query": "Provide SEO analysis for domain strique.io and suggest keyword expansion opportunities.",
  "summary": "This plan will analyze the current SEO performance of strique.io, identify its top-performing and underperforming keywords, benchmark against competitors, and recommend new keyword opportunities for expansion.",
  "steps": [
    {
      "step_id": 1,
      "server": "gsc",
      "step_goal": "Retrieve and review the current organic search performance of strique.io, including traffic, top queries, pages, CTR, and positions.",
      "selected_tool_names": [],
      "notes": "Used fallback tools for server; no category-based match found."
    },
    {
      "step_id": 2,
      "server": "gsc",
      "step_goal": "Identify the main keywords strique.io currently ranks for, their positions, and traffic contribution.",
      "selected_tool_names": [],
      "notes": "Used fallback tools for server; no category-based match found."
    },
    {
      "step_id": 3,
     

/var/folders/l9/vn_czxl100v8f8l1v4d02fx40000gp/T/ipykernel_96890/940612028.py:15: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(json.dumps(tool_selection.dict(), indent=2))


In [3]:
llm = ChatOpenAI(model="gpt-4.1", temperature=0)
seo_tools = SeoTools()
seo_agent = SEOAgent(llm, seo_tools)

user_query = "Provide SEO analysis for domain strique.io and suggest keyword expansion opportunities."

plan = await seo_agent.plan_query(user_query)
tool_selection = await seo_agent.select_tools_for_plan(plan)
code_str = await seo_agent.generate_code_for_query(user_query, plan, tool_selection)

print(code_str)

import asyncio

async def run() -> dict:
    result = {
        "summary": "",
        "steps": []
    }

    # Step 1: Assess overall organic search performance for strique.io
    step1_desc = "Assessed overall organic search performance for strique.io, including traffic, top queries, and landing pages."
    step1_raw = None
    step1_insights = "No tools available for this step. Unable to retrieve organic search performance data for strique.io."
    result["steps"].append({
        "step_id": 1,
        "description": step1_desc,
        "raw_results": step1_raw,
        "key_insights": step1_insights
    })

    # Step 2: Identify current keyword rankings, positions, search volumes, and CTRs
    step2_desc = "Identified current keyword rankings, their positions, search volumes, and click-through rates for strique.io."
    step2_raw = None
    step2_insights = "No tools available for this step. Unable to retrieve keyword rankings and related metrics for strique.io."
    result["steps

In [1]:
from langchain_openai import ChatOpenAI
from src.tools.seo_tools import SeoTools
from src.agents.seo_agent import SEOAgent

llm = ChatOpenAI(model="gpt-4.1", temperature=0)
seo_tools = SeoTools()
seo_agent = SEOAgent(llm, seo_tools)

user_query = "Provide SEO analysis for domain strique.io and suggest keyword expansion opportunities."

result = await seo_agent.run_query_pipeline(user_query)

import json
print(json.dumps(result["plan"], indent=2))               # The LLM plan
print(json.dumps(result["tool_selection"], indent=2))     # Which tools were chosen per step
print(result["code"])                                     # Generated async def run()
print(json.dumps(result["execution"], indent=2))   

{
  "original_query": "Provide SEO analysis for domain strique.io and suggest keyword expansion opportunities.",
  "summary": "This plan will analyze the current SEO performance of strique.io, identify top-performing and underperforming keywords, benchmark competitors, and suggest new keyword opportunities for expansion.",
  "steps": [
    {
      "id": 1,
      "goal": "Retrieve and review current SEO performance metrics for strique.io, including traffic, queries, top pages, CTR, and average positions.",
      "server": "gsc",
      "categories": [
        "gsc_performance",
        "gsc_queries",
        "gsc_pages"
      ],
      "required_inputs": [
        "domain",
        "date_range"
      ],
      "notes": "Start with the last 3-6 months for a comprehensive view."
    },
    {
      "id": 2,
      "goal": "Identify the top and underperforming keywords for strique.io based on impressions, clicks, CTR, and position.",
      "server": "gsc",
      "categories": [
        "gsc_que

In [1]:
from langchain_openai import ChatOpenAI
from src.tools.seo_tools import SeoTools
from src.agents.seo_agent import SEOAgent

llm = ChatOpenAI(model="gpt-4.1", temperature=0)
seo_tools = SeoTools()
seo_agent = SEOAgent(llm, seo_tools)

user_query = "Provide SEO analysis for domain strique.io and suggest keyword expansion opportunities."
plan = await seo_agent.plan_query(user_query)

tool_selection = await seo_agent.select_tools_for_plan(plan)

import json
print(json.dumps(tool_selection.dict(), indent=2))

{
  "original_query": "Provide SEO analysis for domain strique.io and suggest keyword expansion opportunities.",
  "summary": "This plan will analyze the current SEO performance of strique.io, identify top-performing and underperforming keywords, benchmark competitors, and suggest new keyword opportunities for expansion.",
  "steps": [
    {
      "step_id": 1,
      "server": "gsc",
      "step_goal": "Retrieve and review the current SEO performance metrics for strique.io, including traffic, top queries, pages, CTR, and average positions.",
      "selected_tool_names": [
        "list_properties",
        "add_site",
        "delete_site"
      ],
      "notes": "Used fallback tools for server; no category-based match found."
    },
    {
      "step_id": 2,
      "server": "gsc",
      "step_goal": "Identify the top-performing and underperforming keywords and landing pages for strique.io.",
      "selected_tool_names": [
        "list_properties",
        "add_site",
        "delete_

/var/folders/l9/vn_czxl100v8f8l1v4d02fx40000gp/T/ipykernel_3337/2285769655.py:15: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(json.dumps(tool_selection.dict(), indent=2))


In [2]:
from langchain_openai import ChatOpenAI
from src.tools.seo_tools import SeoTools
from src.agents.seo_agent import SEOAgent

llm = ChatOpenAI(model="gpt-4.1", temperature=0)
seo_tools = SeoTools()
seo_agent = SEOAgent(llm, seo_tools)

user_query = "How can I increase the organic keyword coverage for strique.io?"

final_answer = await seo_agent.run_and_respond(user_query)
print(final_answer)


**Overview:**  
strique.io currently has extremely limited organic keyword coverage, with only one top-ranking keyword identified and no additional keyword ideas or related keywords generated. Technical issues prevented a full keyword gap analysis and prioritization.

**Key Findings:**
- strique.io is not present in Google Search Console (GSC), so no performance data is available.
- Only one top-ranking keyword was found for the domain, indicating minimal organic visibility.
- No new keyword ideas or related keywords were generated from DataForSEO, suggesting either a lack of content or insufficient site authority.
- Keyword gap analysis could not be completed due to missing data.
- No prioritized keywords were identified for targeted optimization.

**Recommended Actions:**
- **Verify and Add strique.io to Google Search Console:** Ensure the site is verified in GSC to access performance data and monitor keyword coverage.
- **Expand Site Content:** Create high-quality, relevant content 